In [ ]:
import json
import os
from pathlib import Path

import anthropic
import google.generativeai as genai
import huggingface_hub
import mistralai
import ollama
import openai
from dotenv import load_dotenv
from google.generativeai import GenerativeModel

from phoenix.client import Client
from phoenix.client.utils import to_chat_messages_and_kwargs

load_dotenv(Path.home() / ".env")
genai.configure()

# Instrumentations

In [ ]:
from openinference.instrumentation.anthropic import AnthropicInstrumentor
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.instrumentation.vertexai import VertexAIInstrumentor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.bedrock import BedrockInstrumentor
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:4317"
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

AnthropicInstrumentor().instrument(tracer_provider=tracer_provider)
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)
BedrockInstrumentor().instrument(tracer_provider=tracer_provider)
VertexAIInstrumentor().instrument(tracer_provider=tracer_provider)

# OpenAI

In [ ]:
prompt_version_id = "UHJvbXB0VmVyc2lvbjoy"
prompt_version = Client().prompts.get(prompt_version_id=prompt_version_id)
print(f"prompt_version = {prompt_version.model_dump_json(indent=2)}\n{'-' * 100}")

messages, kwargs = to_chat_messages_and_kwargs(
    prompt_version, variables={"question": "Who made you?"}
)
print(f"messages = {json.dumps(messages, indent=2)}\n{'-' * 100}")
print(f"kwargs = {json.dumps(kwargs, indent=2)}\n{'-' * 100}")

response = openai.OpenAI().chat.completions.create(messages=messages, **kwargs)
print(f"response = {response.model_dump_json(indent=2)}")

# Anthropic

In [ ]:
prompt_version_id = "UHJvbXB0VmVyc2lvbjo0"
prompt_version = Client().prompts.get(prompt_version_id=prompt_version_id)
print(f"prompt_version = {prompt_version.model_dump_json(indent=2)}\n{'-' * 100}")

messages, kwargs = to_chat_messages_and_kwargs(
    prompt_version, variables={"question": "Who made you?"}
)
print(f"messages = {json.dumps(messages, indent=2)}\n{'-' * 100}")
print(f"kwargs = {json.dumps(kwargs, indent=2)}\n{'-' * 100}")

response = anthropic.Anthropic().messages.create(messages=messages, **kwargs)
print(f"response = {response.model_dump_json(indent=2)}")

# Gemini

In [ ]:
prompt_version_id = "UHJvbXB0VmVyc2lvbjo1"
prompt_version = Client().prompts.get(prompt_version_id=prompt_version_id)
print(f"prompt_version = {prompt_version.model_dump_json(indent=2)}\n{'-' * 100}")

message, kwargs = to_chat_messages_and_kwargs(
    prompt_version, variables={"question": "Who made you?"}
)
print(f"messages = {json.dumps(messages, indent=2)}\n{'-' * 100}")
print(f"kwargs = {json.dumps(kwargs, indent=2)}\n{'-' * 100}")

response = GenerativeModel(**kwargs).start_chat(history=messages[:-1]).send_message(messages[-1])
print(f"response = {response}\n")

# Hugging Face Hub

In [ ]:
model = "microsoft/Phi-3.5-mini-instruct"

prompt_version_id = "UHJvbXB0VmVyc2lvbjox"
prompt_version = Client().prompts.get(prompt_version_id=prompt_version_id)
print(f"prompt_version = {prompt_version.model_dump_json(indent=2)}\n{'-' * 100}")

messages, kwargs = to_chat_messages_and_kwargs(
    prompt_version, variables={"question": "Who made you?"}
)
print(f"messages = {json.dumps(messages, indent=2)}\n{'-' * 100}")
print(f"kwargs = {json.dumps(kwargs, indent=2)}\n{'-' * 100}")

response = huggingface_hub.InferenceClient().chat_completion(messages, model=model)
print(f"response = {json.dumps(response, indent=2)}")

# Ollama

In [ ]:
# ollama run hf.co/unsloth/DeepSeek-R1-Distill-Llama-8B-GGUF:Q2_K
model = "hf.co/unsloth/DeepSeek-R1-Distill-Llama-8B-GGUF:Q2_K"

prompt_version_id = "UHJvbXB0VmVyc2lvbjox"
prompt_version = Client().prompts.get(prompt_version_id=prompt_version_id)
print(f"prompt_version = {prompt_version.model_dump_json(indent=2)}\n{'-' * 100}")

messages, kwargs = to_chat_messages_and_kwargs(
    prompt_version, variables={"question": "Who made you?"}
)
print(f"messages = {json.dumps(messages, indent=2)}\n{'-' * 100}")
print(f"kwargs = {json.dumps(kwargs, indent=2)}\n{'-' * 100}")

response = ollama.chat(model=model, messages=messages)
print(f"response = {response.model_dump_json(indent=2)}")

# Mistral AI

In [ ]:
model = "mistral-small-latest"

prompt_version_id = "UHJvbXB0VmVyc2lvbjox"
prompt_version = Client().prompts.get(prompt_version_id=prompt_version_id)
print(f"prompt_version = {prompt_version.model_dump_json(indent=2)}\n{'-' * 100}")

messages, kwargs = to_chat_messages_and_kwargs(
    prompt_version, variables={"question": "Who made you?"}
)
print(f"messages = {json.dumps(messages, indent=2)}\n{'-' * 100}")
print(f"kwargs = {json.dumps(kwargs, indent=2)}\n{'-' * 100}")

response = mistralai.Mistral(api_key=os.environ["MISTRAL_API_KEY"]).chat.complete(
    model="mistral-small-latest", messages=messages
)
print(f"response = {response.model_dump_json(indent=2)}")